# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f2adcace4f0>
├── 'a' --> tensor([[ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875]])
└── 'x' --> <FastTreeValue 0x7f2adcace070>
    └── 'c' --> tensor([[ 0.8068, -0.8125,  0.1606,  0.9079],
                        [-0.5254, -1.5859, -0.7631, -1.6879],
                        [ 0.7782, -0.6509,  1.1106,  1.2761]])

In [4]:
t.a

tensor([[ 1.7414,  0.4205, -1.4305],
        [-0.2482, -0.1569,  0.0875]])

In [5]:
%timeit t.a

76.6 ns ± 0.195 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f2adcace4f0>
├── 'a' --> tensor([[-1.7038,  0.3012, -0.5718],
│                   [-1.0749,  0.0925, -0.2006]])
└── 'x' --> <FastTreeValue 0x7f2adcace070>
    └── 'c' --> tensor([[ 0.8068, -0.8125,  0.1606,  0.9079],
                        [-0.5254, -1.5859, -0.7631, -1.6879],
                        [ 0.7782, -0.6509,  1.1106,  1.2761]])

In [7]:
%timeit t.a = new_value

80.8 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875]]),
    x: Batch(
           c: tensor([[ 0.8068, -0.8125,  0.1606,  0.9079],
                      [-0.5254, -1.5859, -0.7631, -1.6879],
                      [ 0.7782, -0.6509,  1.1106,  1.2761]]),
       ),
)

In [10]:
b.a

tensor([[ 1.7414,  0.4205, -1.4305],
        [-0.2482, -0.1569,  0.0875]])

In [11]:
%timeit b.a

70.9 ns ± 0.0712 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0939, -0.5881, -1.6384],
               [-0.3145,  1.0813, -0.1367]]),
    x: Batch(
           c: tensor([[ 0.8068, -0.8125,  0.1606,  0.9079],
                      [-0.5254, -1.5859, -0.7631, -1.6879],
                      [ 0.7782, -0.6509,  1.1106,  1.2761]]),
       ),
)

In [13]:
%timeit b.a = new_value

591 ns ± 0.808 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

976 ns ± 2.77 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 37.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

168 µs ± 672 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

167 µs ± 906 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2a2f6e8d60>
├── 'a' --> tensor([[[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]],
│           
│                   [[ 1.7414,  0.4205, -1.4305],
│                    [-0.2482, -0.1569,  0.0875]]])
└── 'x' --> <FastTreeValue 0x7f2a2f6e8e50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

37.3 µs ± 78.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f2ab3f2b580>
├── 'a' --> tensor([[ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875],
│                   [ 1.7414,  0.4205, -1.4305],
│                   [-0.2482, -0.1569,  0.0875]])
└── 'x' --> <FastTreeValue 0x7f2ab3f2b700>
    └── 'c' --> tensor([[ 0.8068, -0.8125,  0.1606,  0.9079],
                        [-0.5254, -1.5859, -0.7631, -1.6879],
                 

In [23]:
%timeit t_cat(trees)

35.2 µs ± 179 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.5 µs ± 86.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]],
       
               [[ 1.7414,  0.4205, -1.4305],
                [-0.2482, -0.1569,  0.0875]]]),
    x: Batch(
           c: tensor([[[ 0.8068, -0.8125,  0.1606,  0.9079],
                       [-0.5254, -1.5859, -0.7631, -1.6879],
                       [ 0.7782, -0.6509,  1.1106,  1.2761]],
         

In [26]:
%timeit Batch.stack(batches)

91.2 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875],
               [ 1.7414,  0.4205, -1.4305],
               [-0.2482, -0.1569,  0.0875]]),
    x: Batch(
           c: tensor([[ 0.8068, -0.8125,  0.1606,  0.9079],
                      [-0.5254, -1.5859, -0.7631, -1.6879],
                      [ 0.7782, -0.6509,  1.1106,  1.2761],
                      [ 0.8068, -0.8125,  0.1606,  0.9079],
                      [-0.5254, -

In [28]:
%timeit Batch.cat(batches)

169 µs ± 371 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

377 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
